# X2K API Tutorial Notebook
February 21<sup>st</sup>, 2019

This Jupyter Notebook contains an interactive tutorial for **running the Expression2Kinases (X2K) API** using Python 3.

### Table of Contents
The notebook contains the following sections:
1. **<a href="#1">API Documentation</a>** - shows how to programmatically analyze your gene list in Python.
2. **<a href="#2">Using the X2K API</a>** - overview of the input parameters and output of the API.
3. **<a href="#3">Interpreting the results</a>** - gives an overview of the structure and meaning of the analysis results.
    * **<a href="#chea">Transcription Factor Enrichment Analysis</a>** (ChEA)
    * **<a href="#g2n">Protein-Protein Interaction Expansion</a>** (G2N)
    * **<a href="#kea">Kinase Enrichment Analysis</a>** (KEA)
    * **<a href="#x2k">Expression2Kinases</a>** (X2K)

## 1. <span id="1">Using the X2K API</span>
The X2K API allows for programmatic analysis of an input gene list.

The `run_X2K()` function displayed below can be used to analyze a gene list and load the results in a Python dictionary by performing a **POST request**.

The function requires only one input, `input_genes`, **a list of gene symbols ** to be analyzed. Additional optional parameters can be specified with the `options` parameters.

In [1]:
# Import modules
import http.client
import json


##### Function to run X2K
### Input: a Python list of gene symbols
### Output: a dictionary containing the results of X2K, ChEA, G2N, KEA.

def run_X2K(input_genes, options={}):
    # Open HTTP connection
    conn = http.client.HTTPConnection("amp.pharm.mssm.edu")  #

    # Get default options
    default_options = {'text-genes': '\n'.join(input_genes), 'included_organisms': 'both',
                       'TF-target gene background database used for enrichment': 'ChEA 2015', 'path_length': 2, 'min_network_size': 50,
                       'min_number_of_articles_supporting_interaction': 2,
                       'max_number_of_interactions_per_protein': 200, 'max_number_of_interactions_per_article': 100,
                       'enable_Biocarta': True, 'enable_BioGRID': True, 'enable_DIP': True, 'enable_InnateDB': True, 'enable_IntAct': True, 'enable_KEGG': True,
                       'enable_MINT': True, 'enable_ppid': True, 'enable_SNAVI': True, 'number_of_results': 50,
                       'sort transcription factors by': 'combined score', 'sort kinases by': 'combined score', 'kinase interactions to include': 'ARCHS4'}

    # Update options
    for key, value in options.items():
        if key in default_options.keys() and key != 'text-genes':
            default_options.update({key: value})

    # Get payload
    boundary = "----WebKitFormBoundary7MA4YWxkTrZu0gW"
    payload = ''.join(
        ['--' + boundary + '\r\nContent-Disposition: form-data; name=\"{key}\"\r\n\r\n{value}\r\n'.format(**locals())
         for key, value in default_options.items()]) + '--' + boundary + '--'

    # Get Headers
    headers = {
        'content-type': "multipart/form-data; boundary=" + boundary,
        'cache-control': "no-cache",
    }

    # Initialize connection
    conn.request("POST", "/X2K/api", payload, headers)

    # Get response
    res = conn.getresponse()

    # Read response
    data = res.read().decode('utf-8')

    # Convert to dictionary
    x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in json.loads(data).items()}

    # Clean results
    x2k_results['ChEA'] = x2k_results['ChEA']['tfs']
    x2k_results['G2N'] = x2k_results['G2N']['network']
    x2k_results['KEA'] = x2k_results['KEA']['kinases']
    x2k_results['X2K'] = x2k_results['X2K']['network']

    # Return results
    return x2k_results


In [2]:
# Get input genes
input_genes = ['Nsun3', 'Polrmt', 'Nlrx1', 'Sfxn5', 'Zc3h12c', 'Slc25a39', 'Arsg', 'Defb29', 'Ndufb6', 'Zfand1',
               'Tmem77', '5730403B10Rik', 'Tlcd1', 'Psmc6', 'Slc30a6', 'LOC100047292', 'Lrrc40', 'Orc5l', 'Mpp7',
               'Unc119b', 'Prkaca', 'Tcn2', 'Psmc3ip', 'Pcmtd2', 'Acaa1a', 'Lrrc1', '2810432D09Rik', 'Sephs2', 'Sac3d1',
               'Tmlhe', 'LOC623451', 'Tsr2', 'Plekha7', 'Gys2', 'Arhgef12', 'Hibch', 'Lyrm2', 'Zbtb44', 'Entpd5',
               'Rab11fip2', 'Lipt1', 'Intu', 'Anxa13', 'Klf12', 'Sat2', 'Gal3st2', 'Vamp8', 'Fkbpl', 'Aqp11', 'Trap1',
               'Pmpcb', 'Tm7sf3', 'Rbm39', 'Bri3', 'Kdr', 'Zfp748', 'Nap1l1', 'Dhrs1', 'Lrrc56', 'Wdr20a', 'Stxbp2',
               'Klf1', 'Ufc1', 'Ccdc16', '9230114K14Rik', 'Rwdd3', '2610528K11Rik', 'Aco1', 'Cables1', 'LOC100047214',
               'Yars2', 'Lypla1', 'Kalrn', 'Gyk', 'Zfp787', 'Zfp655', 'Rabepk', 'Zfp650', '4732466D17Rik', 'Exosc4',
               'Wdr42a', 'Gphn', '2610528J11Rik', '1110003E01Rik', 'Mdh1', '1200014M14Rik', 'AW209491', 'Mut',
               '1700123L14Rik', '2610036D13Rik', 'Cox15', 'Tmem30a', 'Nsmce4a', 'Tm2d2', 'Rhbdd3', 'Atxn2', 'Nfs1',
               '3110001I20Rik', 'BC038156', 'LOC100047782', '2410012H22Rik', 'Rilp', 'A230062G08Rik', 'Pttg1ip', 'Rab1',
               'Afap1l1', 'Lyrm5', '2310026E23Rik', 'C330002I19Rik', 'Zfyve20', 'Poli', 'Tomm70a', 'Slc7a6os', 'Mat2b',
               '4932438A13Rik', 'Lrrc8a', 'Smo', 'Nupl2', 'Trpc2', 'Arsk', 'D630023B12Rik', 'Mtfr1', '5730414N17Rik',
               'Scp2', 'Zrsr1', 'Nol7', 'C330018D20Rik', 'Ift122', 'LOC100046168', 'D730039F16Rik', 'Scyl1',
               '1700023B02Rik', '1700034H14Rik', 'Fbxo8', 'Paip1', 'Tmem186', 'Atpaf1', 'LOC100046254', 'LOC100047604',
               'Coq10a', 'Fn3k', 'Sipa1l1', 'Slc25a16', 'Slc25a40', 'Rps6ka5', 'Trim37', 'Lrrc61', 'Abhd3', 'Gbe1',
               'Parp16', 'Hsd3b2', 'Esm1', 'Dnajc18', 'Dolpp1', 'Lass2', 'Wdr34', 'Rfesd', 'Cacnb4', '2310042D19Rik',
               'Srr', 'Bpnt1', '6530415H11Rik', 'Clcc1', 'Tfb1m', '4632404H12Rik', 'D4Bwg0951e', 'Med14', 'Adhfe1',
               'Thtpa', 'Cat', 'Ell3', 'Akr7a5', 'Mtmr14', 'Timm44', 'Sf1', 'Ipp', 'Iah1', 'Trim23', 'Wdr89', 'Gstz1',
               'Cradd', '2510006D16Rik', 'Fbxl6', 'LOC100044400', 'Zfp106', 'Cd55', '0610013E23Rik', 'Afmid', 'Tmem86a',
               'Aldh6a1', 'Dalrd3', 'Smyd4', 'Nme7', 'Fars2', 'Tasp1', 'Cldn10', 'A930005H10Rik', 'Slc9a6', 'Adk',
               'Rbks', '2210016F16Rik', 'Vwce', '4732435N03Rik', 'Zfp11', 'Vldlr', '9630013D21Rik', '4933407N01Rik',
               'Fahd1', 'Mipol1', '1810019D21Rik', '1810049H13Rik', 'Tfam', 'Paics', '1110032A03Rik', 'LOC100044139',
               'Dnajc19', 'BC016495', 'A930041I02Rik', 'Rqcd1', 'Usp34', 'Zcchc3', 'H2afj', 'Phf7', '4921508D12Rik',
               'Kmo', 'Prpf18', 'Mcat', 'Txndc4', '4921530L18Rik', 'Vps13b', 'Scrn3', 'Tor1a', 'AI316807', 'Acbd4',
               'Fah', 'Apool', 'Col4a4', 'Lrrc19', 'Gnmt', 'Nr3c1', 'Sip1', 'Ascc1', 'Fech', 'Abhd14a', 'Arhgap18',
               '2700046G09Rik', 'Yme1l1', 'Gk5', 'Glo1', 'Sbk1', 'Cisd1', '2210011C24Rik', 'Nxt2', 'Notum', 'Ankrd42',
               'Ube2e1', 'Ndufv1', 'Slc33a1', 'Cep68', 'Rps6kb1', 'Hyi', 'Aldh1a3', 'Mynn', '3110048L19Rik', 'Rdh14',
               'Proz', 'Gorasp1', 'LOC674449', 'Zfp775', '5430437P03Rik', 'Npy', 'Adh5', 'Sybl1', '4930432O21Rik',
               'Nat9', 'LOC100048387', 'Mettl8', 'Eny2', '2410018G20Rik', 'Pgm2', 'Fgfr4', 'Mobkl2b', 'Atad3a',
               '4932432K03Rik', 'Dhtkd1', 'Ubox5', 'A530050D06Rik', 'Zdhhc5', 'Mgat1', 'Nudt6', 'Tpmt', 'Wbscr18',
               'LOC100041586', 'Cdk5rap1', '4833426J09Rik', 'Myo6', 'Cpt1a', 'Gadd45gip1', 'Tmbim4', '2010309E21Rik',
               'Asb9', '2610019F03Rik', '7530414M10Rik', 'Atp6v1b2', '2310068J16Rik', 'Ddt', 'Klhdc4', 'Hpn', 'Lifr',
               'Ovol1', 'Nudt12', 'Cdan1', 'Fbxo9', 'Fbxl3', 'Hoxa7', 'Aldh8a1', '3110057O12Rik', 'Abhd11', 'Psmb1',
               'ENSMUSG00000074286', 'Chpt1', 'Oxsm', '2310009A05Rik', '1700001L05Rik', 'Zfp148', '39509', 'Mrpl9',
               'Tmem80', '9030420J04Rik', 'Naglu', 'Plscr2', 'Agbl3', 'Pex1', 'Cno', 'Neo1', 'Asf1a', 'Tnfsf5ip1',
               'Pkig', 'AI931714', 'D130020L05Rik', 'Cntd1', 'Clec2h', 'Zkscan1', '1810044D09Rik', 'Mettl7a', 'Siae',
               'Fbxo3', 'Fzd5', 'Tmem166', 'Tmed4', 'Gpr155', 'Rnf167', 'Sptlc1', 'Riok2', 'Tgds', 'Pms1', 'Pitpnc1',
               'Pcsk7', '4933403G14Rik', 'Ei24', 'Crebl2', 'Tln1', 'Mrpl35', '2700038C09Rik', 'Ubie', 'Osgepl1',
               '2410166I05Rik', 'Wdr24', 'Ap4s1', 'Lrrc44', 'B3bp', 'Itfg1', 'Dmxl1', 'C1d']

# Run X2K results
x2k_results = run_X2K(input_genes)
x2k_results.keys()

dict_keys(['X2K', 'ChEA', 'KEA', 'G2N', 'input'])

## 2. <span id="2">X2K API Documentation</span>

### 2.1 API Inputs
A **full list of the input parameters** for the `run_X2K()` function is available below.

The optional parameters can provided to the function in the `options` dictionary.

<table id="parameter-table">
    <tr>
        <th style='text-align=left;'>Parameter</th>
        <th style='text-align=left;'>Step</th>
        <th style='text-align=left;'>Description</th>
        <th style='text-align=left;'>Notes</th>
    </tr>

    <tr>
        <td style='text-align=left;'>**input_genes** (required)</td>
        <td style='text-align=left;'>X2K</td>
        <td style='text-align=left;'>Contains the input gene set for the X2K analysis.</td>
        <td style='text-align=left;'>A list of strings representing the input gene symbols.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*organism* (optional)</td>
        <td style='text-align=left;'>ChEA</td>
        <td style='text-align=left;'>The organism from which TF-target interaction data should be integrated.</td>
        <td style='text-align=left;'>One of `('human_only', 'mouse_only', 'both')`. Default `'both'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*TF-target gene background database used for enrichment* (optional)</td>
        <td style='text-align=left;'>ChEA</td>
        <td style='text-align=left;'>The database from which TF-target interaction data should be integrated,</td>
        <td style='text-align=left;'>One of `('ChEA 2015', 'ENCODE 2015', 'ChEA & ENCODE Consensus', 'TRANSFAC & JASPAR', 'ChEA 2016', 'ARCHS4', 'CREEDS', 'Enrichr Co-occurrence')` Default `'ChEA & ENCODE Consensus')`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*sort transcription factors by* (optional) 
        <td style='text-align=left;'>ChEA</td>
        <td style='text-align=left;'>The method used to sort the top Transcription Factors identified by ChEA.</td>
        <td style='text-align=left;'>One of `('p-value', 'rank', 'combined score')`. Default `'p-value'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*path_length* (optional)</td>
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The maximum Protein-Protein Interaction path length for the network expansion.</td>
        <td style='text-align=left;'>Integer, default `2`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*minimum_network_size* (optional)
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The minimum size of the Protein-Protein interaction network generated using Genes2Networks.</td>
        <td style='text-align=left;'>Integer, default `50`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*min_number_of_articles_supporting_interaction* (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The minimum number of published articles supporting a Protein-Protein Interaction for the expanded subnetwork.</td>
        <td style='text-align=left;'>Integer, default `2`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*max_number_of_interactions_per_protein* (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The maximum number of physical interactions allowed for the proteins in the expanded subnetwork.</td>
        <td style='text-align=left;'>Integer, default `200`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*max_number_of_interactions_per_article* (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The maximum number of physical interactions reported in each published article</td>
        <td style='text-align=left;'>Integer, default `100`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_Biocarta (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_BioGRID (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'true'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_BioPlex (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_DIP (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_huMAP (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_InnateDB (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_IntAct (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'true'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_KEGG (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_MINT (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'true'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_ppid (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'true'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_SNAVI (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_iREF (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_Stelzl (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'true'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_vidal (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_BIND (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_figeys (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>enable_HPRD (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The Protein-Protein Interaction databases to integrate for generation of the expanded subnetwork.</td>
        <td style='text-align=left;'>Either `'true'` or `'false'`. Default `'false'`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*number_of_results* (optional) 
        <td style='text-align=left;'>G2N</td>
        <td style='text-align=left;'>The maximum network size of the expanded network generated using Genes2Networks.</td>
        <td style='text-align=left;'>Integer, default `50`.</td>
    </tr>
    <tr>
        <td style='text-align=left;'>*kinase interactions to include* 
        <td style='text-align=left;'>KEA</td>
        <td style='text-align=left;'>Kinase interactions databases to include.</td>
        <td style='text-align=left;'>One of `('p-value', 'rank', 'combined score')`. Default `'p-value'`.</td>
    </tr>        
    <tr>
        <td style='text-align=left;'>*sort_kinases_by* (optional) 
        <td style='text-align=left;'>KEA</td>
        <td style='text-align=left;'>The method used to sort the top Transcription Factors identified by KEA.</td>
        <td style='text-align=left;'>One of `('kea 2018', 'ARCHS4', 'iPTMnet', 'NetworkIN', 'Phospho.ELM', 'Phosphopoint', 'PhosphoPlus', 'MINT')`. Default `'kea 2018'`.</td>
    </tr>
</table>


### 2.2 API Output
The `run_X2K()` function returns results as `dict` containing **four keys**, whose contents are described below.

<table id="parameter-table">
    <tr>
        <th style='text-align: left;'>Key</th>
        <th style='text-align: left;'>Notes</th>
        <th style='text-align: left;'>Contents</th>
    </tr>

    <tr>
        <td style='text-align: left;'>**ChEA**</td>
        <td style='text-align: left;'>Contains the results of the **Transcription Factor Enrichment Analysis**, generated using ChEA.</td>
        <td style='text-align: left;'>A `list` of `dict`s containing information on the top TFs predicted to regulate the input genes. </td>
    </tr>
 
    <tr>
        <td style='text-align: left;'>**G2N**</td>
        <td style='text-align: left;'>Contains the results of the **Protein-Protein Interaction Expansion**, generated using Genes2Networks (G2N).</td>
        <td style='text-align: left;'>A `dict` containing two keys:
            <ul>
                <li><b>nodes</b>: A `list` containing information on the nodes of the expanded subnetwork.</li>
                <li><b>interactions</b>: A `list` containing information on the edges of the expanded subnetwork.</li>
            </ul>
        </td>
    </tr>
 
    <tr>
        <td style='text-align: left;'>**KEA**</td>
        <td style='text-align: left;'>Contains the results of the **Kinase Enrichment Analysis**, generated using KEA.</td>
        <td style='text-align: left;'>A `list` of `dict`s containing information on the top kinases predicted to regulate the subnetwork identified by G2N. </td>
    </tr>
 
    <tr>
        <td style='text-align: left;'>**X2K**</td>
        <td style='text-align: left;'>Contains the **Expression2Kinases network**, generated by integrating the results of ChEA, G2N and KEA.</td>
        <td style='text-align: left;'>A `dict` containing two keys:
            <ul>
                <li><b>nodes</b>: A `list` containing information on the nodes of the final X2K network.</li>
                <li><b>interactions</b>: A `list` containing information on the edges of the final X2K network.</li>
            </ul>
        </td>
    </tr>
 
</table>


## 3. <span id="3">Interpreting the Results</span>

### 3.1 <span id="chea">ChEA results</span>
The results for the ChEA analysis can be accessed in <code>x2k_results['ChEA']</code>. Here, the results are converted to a pandas DataFrame for easier interpretation.

In [3]:
# Import pandas
import pandas as pd

# Read results
chea_dataframe = pd.DataFrame(x2k_results['ChEA'])
chea_dataframe.head()

,combinedScore,enrichedTargets,meta,name,pvalue,simpleName,zscore
0,0.0,"[D4BWG0951E, AW209491, RDH14, SFXN5, SMO, AP4S...",{},E2F1_18555785_CHIP-SEQ_MESC_MOUSE,6.463489e-54,E2F1,0.0
1,0.0,"[PRKACA, SF1, PKIG, SMO, NEO1, VPS13B, C1D, CA...",{},CREM_20920259_CHIP-SEQ_GC1-SPG_MOUSE,1.086405e-31,CREM,0.0
2,0.0,"[AW209491, SF1, RDH14, SMO, AP4S1, SAC3D1, ZKS...",{},ZFX_18555785_CHIP-SEQ_MESC_MOUSE,2.027806e-31,ZFX,0.0
3,0.0,"[MTMR14, 1700023B02RIK, TSR2, 2010309E21RIK, S...",{},JARID1A_20064375_CHIP-SEQ_MESC_MOUSE,1.385773e-29,JARID1A,0.0
4,0.0,"[AW209491, SAC3D1, 1810049H13RIK, GSTZ1, RFESD...",{},MYC_18358816_CHIP-CHIP_MESC_MOUSE,3.823788e-29,MYC,0.0


** Table 1 | Results of the ChEA analysis. ** Each row represents a transcription factor predicted to regulate the input gene list.

### 3.2 <span id="g2n">G2N Results</span>
The results for the G2N analysis can be accessed in <code>x2k_results['G2N']</code>.

The results are stored in a dictionary containing two keys:
* `edges`
* `interactions`

In [4]:
# G2N nodes dataframe
g2n_nodes_dataframe = pd.DataFrame(x2k_results['G2N']['nodes']).drop('pvalue', axis=1)
g2n_nodes_dataframe.head()

,name,type
0,E2F1_18555785_CHIP-SEQ_MESC_MOUSE,tf
1,CREM_20920259_CHIP-SEQ_GC1-SPG_MOUSE,tf
2,ZFX_18555785_CHIP-SEQ_MESC_MOUSE,tf
3,JARID1A_20064375_CHIP-SEQ_MESC_MOUSE,tf
4,MYC_18358816_CHIP-CHIP_MESC_MOUSE,tf


** Table 2 | Nodes of the Genes2Networks expanded subnetwork. ** Each row represents a node in the expanded subnetwork. The type column indicates whether the node is a Transcription Factor identified by ChEA, or an intermediate protein.

In [5]:
# G2N edges dataframe
g2n_edges_dataframe = pd.DataFrame(x2k_results['G2N']['interactions'])
g2n_edges_dataframe.head()

,source,target
0,10,10
1,10,255
2,10,90
3,10,218
4,10,56


** Table 3 | Edges of the Genes2Networks expanded subnetwork. ** Each row represents an edge in the expanded subnetwork generated by G2N on the top transcription factors identified by ChEA.

### 3.3 <span id="kea">KEA Results</span>
The results for the KEA analysis can be accessed in <code>x2k_results['KEA']</code>.

In [6]:
# KEA Results
kea_dataframe = pd.DataFrame(x2k_results['KEA'])
kea_dataframe.head()

,combinedScore,enrichedSubstrates,name,pvalue,zscore
0,0.0,"[SMARCA4, NEDD4, ABL1, CTNNB1]",MAP4K4,0.000393,0.0
1,0.0,"[NCOR2, HUWE1, HCFC1]",TRRAP,0.000847,0.0
2,0.0,"[HSPA4, DDB1, NEDD4]",PRKCI,0.004609,0.0
3,0.0,"[TRRAP, ELK1, CARM1]",MAP3K4,0.004609,0.0
4,0.0,"[PML, ETV6, STAT6]",RIPK1,0.004609,0.0


** Table 4 | Results of the KEA analysis. ** Each row represents a protein kinase predicted to regulate the expanded subnetwork generated by G2N.

### 3.4 <span id="x2k">X2K Results</span>
The results for the X2K analysis can be accessed in <code>x2k_results['X2K']</code>.

The results are stored in a dictionary containing two keys:
* `nodes`
* `interactions`

In [7]:
# X2K nodes dataframe
x2k_nodes_dataframe = pd.DataFrame(x2k_results['X2K']['nodes']).drop('pvalue', axis=1)
x2k_nodes_dataframe.head()

,name,type
0,E2F1_18555785_CHIP-SEQ_MESC_MOUSE,tf
1,CREM_20920259_CHIP-SEQ_GC1-SPG_MOUSE,tf
2,ZFX_18555785_CHIP-SEQ_MESC_MOUSE,tf
3,JARID1A_20064375_CHIP-SEQ_MESC_MOUSE,tf
4,MYC_18358816_CHIP-CHIP_MESC_MOUSE,tf


** Table 5 | Nodes of the final Expression2Kinases network. ** Each row represents a node in the final X2K network network. The type column indicates whether the node is a Transcription Factor identified by ChEA, an intermediate protein identified by G2N, or a protein kinase identified by KEA.

In [8]:
# X2K edges dataframe
x2k_edges_dataframe = pd.DataFrame(x2k_results['X2K']['interactions'])
x2k_edges_dataframe.head()

,source,target
0,10,22
1,10,164
2,10,264
3,10,171
4,36,116


** Table 6 | Edges of the final Expression2Kinases subnetwork. ** Each row represents an edge in the final network identified by integrating the results of ChEA, G2N, and KEA.